In [1]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import string
import gensim.downloader as api
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

2024-03-11 15:49:11.778778: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 15:49:11.808103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 15:49:11.808130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 15:49:11.808863: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 15:49:11.813821: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

In [2]:
df=pd.read_csv('one_output_dataset_new.csv')
df

,speech,emotion
0,From the moment that the French defenses at Se...,trust
1,"Five score years ago, a great American, in who...",anticipation
2,The media will not show the magnitude of this ...,anger
3,"Madam Vice President, my fellow Americans: to ...",disgust
4,"My German countrymen, men and women, (long pau...",anger
5,We observe today not a victory of party but a ...,joy
6,"Your Majesties, Your Highnesses, Distinguished...",trust
7,I am honored to be with you today at your comm...,anticipation
8,"Honorable UN Secretary General Mr Ban Ki-moon,...",sadness
9,It is with a profound sense of humility that I...,sadness


In [3]:
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
wv=api.load('word2vec-google-news-300')

In [5]:
# wv.save('vectors.kv')

In [6]:
def sentence_vec(sentence):
    vector_size=wv.vector_size
    wv_res=np.zeros(vector_size)
    ctr=1
    for w in sentence:
        if w in wv:
            ctr+=1
            wv_res+=wv[w]
        wv_res=wv_res/ctr
        return wv_res

In [7]:
def spacy_tokenizer(sentence):
    #creating token object, which is used to create documents with linguistic annotations
    doc=nlp(sentence)
    #lemmatizing each token and lowercase conversion
    mytokens=[word.lemma_.lower().strip() for word in doc]
    # removing stop words
    mytokens=[word for word in mytokens if word not in stop_words and word not in punctuations]
    return mytokens

In [8]:
nlp=spacy.load("en_core_web_sm")
stop_words=nlp.Defaults.stop_words
punctuations=string.punctuation

In [9]:
df['tokens']=df['speech'].apply(spacy_tokenizer)

In [10]:
df['vec']=df['tokens'].apply(sentence_vec)

In [11]:
df

,speech,emotion,tokens,vec
0,From the moment that the French defenses at Se...,trust,"[moment, french, defense, sedan, meuse, break,...","[0.08154296875, 0.0135498046875, 0.072265625, ..."
1,"Five score years ago, a great American, in who...",anticipation,"[score, year, ago, great, american, symbolic, ...","[-0.006866455078125, -0.037109375, 0.008666992..."
2,The media will not show the magnitude of this ...,anger,"[medium, magnitude, crowd, turn, today, look, ...","[0.01373291015625, 0.0498046875, -0.0698242187..."
3,"Madam Vice President, my fellow Americans: to ...",disgust,"[madam, vice, president, fellow, americans, st...","[-0.0002727508544921875, -0.04736328125, -0.01..."
4,"My German countrymen, men and women, (long pau...",anger,"[german, countryman, man, woman, long, pause, ...","[0.033203125, 0.1474609375, -0.0233154296875, ..."
5,We observe today not a victory of party but a ...,joy,"[observe, today, victory, party, celebration, ...","[-0.05517578125, -0.0069580078125, 0.056884765..."
6,"Your Majesties, Your Highnesses, Distinguished...",trust,"[majesties, highnesses, distinguished, guests,...","[-0.045166015625, 0.05517578125, 0.02416992187..."
7,I am honored to be with you today at your comm...,anticipation,"[honor, today, commencement, fine, university,...","[-0.0208740234375, 0.0228271484375, 0.1328125,..."
8,"Honorable UN Secretary General Mr Ban Ki-moon,...",sadness,"[honorable, un, secretary, general, mr, ban, k...","[0.02392578125, 0.061767578125, 0.012878417968..."
9,It is with a profound sense of humility that I...,sadness,"[profound, sense, humility, accept, honor, cho...","[0.11181640625, -0.0277099609375, 0.0140991210..."


In [12]:
len(df['vec'][2])

300

In [13]:
speech_vectors=df['vec'].values
labels=df['emotion'].values

In [14]:
labels

array(['trust', 'anticipation', 'anger', 'disgust', 'anger', 'joy',
       'trust', 'anticipation', 'sadness', 'sadness', 'fear', 'joy',
       'anticipation', 'trust'], dtype=object)

In [15]:
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

In [16]:
labels

array([6, 1, 0, 2, 0, 4, 6, 1, 5, 5, 3, 4, 1, 6])

In [17]:
speech_vectors.shape[0]

14

In [18]:

X_train,X_test,y_train,y_test = train_test_split(speech_vectors,labels,test_size=0.2,random_state=7)

In [19]:
Y_test=y_test

In [20]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [21]:
X_train=X_train.tolist()
X_test=X_test.tolist()
y_train=y_train.tolist()
y_test=y_test.tolist()

In [22]:

X_train=tf.convert_to_tensor(X_train)
X_test=tf.convert_to_tensor(X_test)
y_train=tf.convert_to_tensor(y_train)
y_test=tf.convert_to_tensor(y_test)

2024-03-11 15:49:53.419873: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-11 15:49:53.421559: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [23]:
y_train

<tf.Tensor: shape=(11, 1), dtype=float32, numpy=
array([[4.],
       [6.],
       [1.],
       [6.],
       [5.],
       [1.],
       [3.],
       [2.],
       [6.],
       [5.],
       [0.]], dtype=float32)>

In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(10000).batch(2)

In [25]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.shuffle(10000).batch(2)

In [26]:
y_test

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[1.],
       [4.],
       [0.]], dtype=float32)>

In [27]:
opt = keras.optimizers.Adam(learning_rate=0.00001)

In [28]:
model = Sequential()
model.add(Embedding(input_dim=speech_vectors.shape[0], output_dim=300, trainable=False))
model.add(LSTM(units=300))
model.add(Dense(units=7, activation='sigmoid'))
model.compile(optimizer=opt, loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [29]:
model.fit(train_dataset, validation_data=test_dataset, epochs=10, batch_size=2)

Epoch 1/10
6/6 [==============================] - 4s 401ms/step - loss: 1.9459 - accuracy: 0.1818 - val_loss: 1.9419 - val_accuracy: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 2s 311ms/step - loss: 1.9448 - accuracy: 0.2727 - val_loss: 1.9431 - val_accuracy: 0.0000e+00
Epoch 3/10
6/6 [==============================] - 2s 309ms/step - loss: 1.9431 - accuracy: 0.2727 - val_loss: 1.9442 - val_accuracy: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 2s 336ms/step - loss: 1.9423 - accuracy: 0.2727 - val_loss: 1.9448 - val_accuracy: 0.0000e+00
Epoch 5/10
6/6 [==============================] - 2s 313ms/step - loss: 1.9414 - accuracy: 0.2727 - val_loss: 1.9455 - val_accuracy: 0.0000e+00
Epoch 6/10
6/6 [==============================] - 2s 333ms/step - loss: 1.9404 - accuracy: 0.2727 - val_loss: 1.9457 - val_accuracy: 0.0000e+00
Epoch 7/10
6/6 [==============================] - 2s 313ms/step - loss: 1.9396 - accuracy: 0.2727 - val_loss: 1.9464 - val_accuracy: 0.0

In [38]:
X_test

<tf.Tensor: shape=(3, 300), dtype=float64, numpy=
array([[-0.04516602,  0.05517578,  0.02416992,  0.07421875, -0.03417969,
        -0.13476562,  0.00430298, -0.09863281,  0.0402832 ,  0.15917969,
         0.08154297, -0.03979492, -0.02282715,  0.07958984, -0.00735474,
        -0.03393555,  0.05908203,  0.01281738,  0.03393555,  0.06103516,
         0.15136719, -0.00084686,  0.04272461, -0.01074219,  0.0559082 ,
        -0.04296875, -0.03613281,  0.05175781, -0.02172852, -0.08642578,
        -0.09472656, -0.09667969, -0.02075195,  0.15429688, -0.15722656,
        -0.05688477,  0.1796875 , -0.07177734,  0.0480957 , -0.03222656,
         0.03149414, -0.03271484,  0.0859375 ,  0.01519775,  0.20214844,
        -0.03662109,  0.00976562,  0.04321289, -0.140625  , -0.06054688,
        -0.17382812,  0.02929688,  0.11279297,  0.1171875 , -0.03369141,
         0.06103516, -0.19824219, -0.04760742,  0.00946045, -0.06298828,
         0.05883789,  0.01348877,  0.13085938, -0.05615234, -0.00878906,
 

In [39]:
y_pred = model.predict(X_test)
# y_pred = np.argmax(y_pred,axis=1)

1/1 [==============================] - 0s 37ms/step


In [40]:
y_pred

array([[0.49449053, 0.51131207, 0.4892127 , 0.48533365, 0.493694  ,
        0.5095649 , 0.5043288 ],
       [0.49449053, 0.51131207, 0.4892127 , 0.48533365, 0.493694  ,
        0.5095649 , 0.5043288 ],
       [0.49449053, 0.51131207, 0.4892127 , 0.48533365, 0.493694  ,
        0.5095649 , 0.5043288 ]], dtype=float32)

In [32]:
print(confusion_matrix([1,4,0], y_pred))
print(classification_report([1,4,0], y_pred))

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       1.0
           6       0.00      0.00      0.00       0.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arjun/miniconda3/envs/ml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arjun/